### Introdução


Neste projeto, daremos início à análise dos dados aplicando dois métodos principais: o **índice de Jaccard** e o **filtro TPEI**.

- Aplicaremos o **índice de Jaccard**, que irá analisar a similaridade entre disciplinas com base em suas recomendações, o cálculo é feito pela razão do tamanho da intersecção com o tamanho da união dos nossos dados que resulta em um **score** de 0 a 1. Para isso, será construído um **grafo direcionado**, em que cada vértice representa uma disciplina e as arestas indicam recomendações entre elas, com os conjuntos comparados sendo as arestas de dois nós diferentes.

- O **filtro TPEI** será responsável por eliminar uma grande quantidade de dados irrelevantes, mantendo apenas as equivalências que ocorrem com valores de TPEI iguais ou superiores. Esse filtro garente que a equivalência entre disciplinas seja significativa quantitativamente.




### Bibliografia utilizada


https://huggingface.co/spaces/Ruchin/jaccard_similarity

https://en.wikipedia.org/wiki/Jaccard_index

https://networkx.org/documentation/stable/reference/classes/digraph.html

### Bibliotecas necessárias para executar o código


import csv

import numpy as np

import pandas as pd

import networkx as nx

import unicodedata

import requests

from io import StringIO

from node2vec import Node2Vec

from sklearn.metrics.pairwise import cosine_similarity

import seaborn as sns

import matplotlib.pyplot as plt

###  Limpeza de Dados e Carregamento do Catálogo


In [ ]:
# Normaliza uma string removendo acentos e espaços extras
def normalize_str(s: str) -> str:
    return (
        unicodedata.normalize('NFKD', s)
        .encode('ASCII', 'ignore')
        .decode('utf-8')
        .strip()
    )

# Versão específica para nomes de disciplinas (já normaliza e coloca lowercase)
def normalizar_nome(nome: str) -> str:
    return normalize_str(nome).lower()

# Baixa o catálogo da UFABC direto do Repositório no GitHub
def carregar_catalogo():
    url = "https://raw.githubusercontent.com/angeloodr/disciplinas-ufabc/main/catalogo_disciplinas_graduacao_2024_2025.tsv"
    print("🔄 Baixando catálogo de disciplinas...")
    resp = requests.get(url)
    resp.raise_for_status()
    df = pd.read_csv(StringIO(resp.text), sep='\t')

    # Normaliza os nomes das colunas
    df.columns = [
        normalize_str(col).upper().replace(' ', '_')
        for col in df.columns
    ]
    print("✅ Download bem-sucedido!")
    print("📝 Colunas disponíveis:", df.columns.tolist())
    return df


### Construção do Grafo de Pré-Requisitos


In [ ]:
#  Constrói grafo com arestas de pré-requisito com base na coluna RECOMENDACAO
def construir_grafo(df: pd.DataFrame) -> nx.DiGraph:
    print("📌 Construindo grafo de pré-requisitos...")
    G = nx.DiGraph()
    total_arestas = 0

    # Cria um dicionário de nome normalizado → sigla
    mapping = {
        normalizar_nome(row['DISCIPLINA']): row['SIGLA']
        for _, row in df.iterrows()
        if pd.notna(row['SIGLA'])
    }

    # Adiciona todos os nós com metadados
    for _, row in df.iterrows():
        sigla = row['SIGLA']
        nome = row['DISCIPLINA']
        if pd.isna(sigla): 
            continue
        G.add_node(sigla, nome=nome)

    # Adiciona as arestas baseadas nas recomendações
    for _, row in df.iterrows():
        curso = row['SIGLA']
        if pd.isna(curso):
            continue

        recs = row.get('RECOMENDACAO', '')
        if pd.isna(recs) or not isinstance(recs, str):
            continue

        for rec in recs.split(';'):
            rec = rec.strip()
            if not rec:
                continue
            rec_norm = normalizar_nome(rec)
            if rec_norm in mapping:
                prereq = mapping[rec_norm]
                if not G.has_edge(prereq, curso):
                    G.add_edge(prereq, curso, tipo='pre_requisito')
                    total_arestas += 1

    print(f"✅ Grafo criado com {G.number_of_nodes()} nós e {total_arestas} arestas.")
    return G


### Processamento do Grafo: Rede Direcionada e Embeddings Estruturais

In [ ]:
#  Remove ciclos do grafo para que seja um DAG (necessário para cálculo de profundidade)
def remover_ciclos(G: nx.DiGraph) -> nx.DiGraph:
    print("🔁 Removendo ciclos (modo eficiente)...")
    G_copy = G.copy()
    removidos = 0
    try:
        while True:
            ciclo = nx.find_cycle(G_copy, orientation='original')
            if ciclo:
                # Remove a primeira aresta do ciclo
                u, v, _ = ciclo[0]
                G_copy.remove_edge(u, v)
                removidos += 1
    except nx.NetworkXNoCycle:
        pass  # Não há mais ciclos

    print(f"✅ Ciclos removidos: {removidos}")
    return G_copy

# Gera embeddings estruturais com Node2Vec
def gerar_embeddings_node2vec(G: nx.DiGraph, dimensions=64, walk_length=10, num_walks=50, workers=1):
    print("🔄 Gerando embeddings estruturais (Node2Vec)...")
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=walk_length, num_walks=num_walks, workers=workers)
    model = node2vec.fit(window=5, min_count=1, batch_words=4)
    embeddings = {node: model.wv[node] for node in G.nodes()}
    print("✅ Embeddings gerados!")
    return embeddings

# Calcula profundidade de cada nó no grafo (camada curricular)
def calcular_profundidade(G: nx.DiGraph):
    print("📏 Calculando profundidade no grafo (DAG)...")
    profundidades = {}
    for node in nx.topological_sort(G):
        preds = list(G.predecessors(node))
        if not preds:
            profundidades[node] = 0
        else:
            profundidades[node] = max(profundidades[p] for p in preds) + 1
    print("✅ Profundidade calculada!")
    return profundidades

# Salva os embeddings em CSV
def salvar_embeddings_csv(embeddings, caminho="embeddings_node2vec.csv"):
    df = pd.DataFrame.from_dict(embeddings, orient='index')
    df.index.name = 'DISCIPLINA'
    df.to_csv(caminho)
    print(f"💾 Embeddings salvos em: {caminho}")

# Fluxo principal
if __name__ == "__main__":
    # Etapa 1 - Carregar catálogo
    df = carregar_catalogo()

    # Etapa 2 - Construir grafo
    grafo = construir_grafo(df)
    nx.write_graphml(grafo, "grafo_pre_requisitos.graphml")
    print("📁 Arquivo salvo: grafo_pre_requisitos.graphml")

    # Etapa 3 - Gerar embeddings estruturais
    embeddings = gerar_embeddings_node2vec(grafo)
    salvar_embeddings_csv(embeddings)

    # Etapa 4 - Remover ciclos e calcular profundidade
    grafo_sem_ciclos = remover_ciclos(grafo)
    profundidades = calcular_profundidade(grafo_sem_ciclos)

    # Etapa 5 - Salvar profundidades em TXT
    with open("profundidade_nos.txt", "w") as f:
        for node, prof in profundidades.items():
            f.write(f"{node}: {prof}\n")
    print("📁 Profundidades salvas em: profundidade_nos.txt")

#### Exemplo de output do profundidade_nos.txt


NHZ2013-11: 1 
ESHR027-21: 1

NHBQ001-22: 2
ESTO013-17: 2

ESHC016-17: 3
ESEC003-24: 3

NHBQ019-22: 4
ESTO015-17: 4

MCTD007-18: 5
MCBM013-23: 5

ESZE026-17: 6
ESZE088-17: 6

MCZB020-13: 7
NHZ1079-15: 7

ESZE079-17: 8
ESTE031-17: 8

ESEN003-23: 9
ESTE019-17: 9

ESZE076-17: 10
ESZE073-17: 10

ESTE033-17: 11

ESZE103-17: 12
ESEN009-23: 12

### Com o grafo pronto e os dados carregados, aplicaremos o Índice de Jaccard e outras medidas de similaridade para analisar as conexões entre as disciplinas.


In [ ]:
# Jaccard entre dois conjuntos
def jaccard_similarity(set1, set2):
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

# Aplica Jaccard entre predecessores ou sucessores
def similaridade_jaccard(G, a, b, tipo='predecessor'):
    try:
        if tipo == 'predecessor':
            set_a = set(G.predecessors(a))
            set_b = set(G.predecessors(b))
        else:
            set_a = set(G.successors(a))
            set_b = set(G.successors(b))
        return jaccard_similarity(set_a, set_b)
    except nx.NetworkXError:
        return 0.0


### Aplicando similaridade de profundidade


In [ ]:
# Similaridade de profundidade
def similaridade_profundidade(prof, a, b):
    if a not in prof or b not in prof:
        return 0.0
    max_p = max(prof.values())
    if max_p == 0:
        return 1.0
    return 1.0 - abs(prof[a] - prof[b]) / max_p

### Aplicando similaridade Node2Vec (cosseno)


In [ ]:
# Similaridade por Node2Vec (cosseno)
def similaridade_node2vec(embeddings, a, b):
    if a not in embeddings.index or b not in embeddings.index:
        return 0.0
    vec_a = embeddings.loc[a].values.reshape(1, -1)
    vec_b = embeddings.loc[b].values.reshape(1, -1)
    return cosine_similarity(vec_a, vec_b)[0][0]

### Combinação das Similaridades em um Score Final

In [ ]:
# Combina múltiplas similaridades em score final
def similaridade_combinada(G, embeddings, profundidades, a, b, pesos=None):
    if pesos is None:
        pesos = {
            'jaccard_pred': 1.0,
            'jaccard_succ': 1.0,
            'profundidade': 1.0,
            'node2vec': 1.0,
        }

    sim_jaccard_pred = similaridade_jaccard(G, a, b, tipo='predecessor')
    sim_jaccard_succ = similaridade_jaccard(G, a, b, tipo='successor')
    sim_profundidade = similaridade_profundidade(profundidades, a, b)
    sim_node2vec = similaridade_node2vec(embeddings, a, b)

    total_peso = sum(pesos.values())
    score = (
        pesos['jaccard_pred'] * sim_jaccard_pred +
        pesos['jaccard_succ'] * sim_jaccard_succ +
        pesos['profundidade'] * sim_profundidade +
        pesos['node2vec'] * sim_node2vec
    ) / total_peso

    return {
        "score_combinado": score,
        "jaccard_pred": sim_jaccard_pred,
        "jaccard_succ": sim_jaccard_succ,
        "profundidade": sim_profundidade,
        "node2vec": sim_node2vec
    }

### Aplicando o Filtro TPEI

In [ ]:
# Filtro TPEI exato: compara atributos T, P, E, I
def filtro_tpei_exato(G, a, b):
    for campo in ['T', 'P', 'E', 'I']:
        if G.nodes[a].get(campo) != G.nodes[b].get(campo):
            return False
    return True

###  Geração do arquivo de similaridades filtradas (.tsv)

In [ ]:
# Escrita incremental com filtro de profundidade e TPEI
if __name__ == "__main__":
    grafo = carregar_grafo()
    embeddings = carregar_embeddings()
    profundidades = carregar_profundidades()

    caminho_saida = "similaridades_disciplinas_filtrado.tsv"
    with open(caminho_saida, mode="w", newline='', encoding='utf-8') as f_out:
        writer = csv.writer(f_out, delimiter="\t")
        # Cabeçalho
        writer.writerow([
            "disciplina_a", "disciplina_b", "score_combinado",
            "jaccard_pred", "jaccard_succ", "profundidade", "node2vec"
        ])

        # Laço otimizado com filtros
        for a in grafo.nodes():
            for b in grafo.nodes():
                if a == b:
                    continue

                # Filtro por profundidade (ex: 2 níveis no máximo)
                if abs(profundidades.get(a, 0) - profundidades.get(b, 0)) > 2:
                    continue

                # Filtro TPEI exato
                if not filtro_tpei_exato(grafo, a, b):
                    continue

                sim = similaridade_combinada(grafo, embeddings, profundidades, a, b)
                writer.writerow([
                    a, b, sim["score_combinado"],
                    sim["jaccard_pred"], sim["jaccard_succ"],
                    sim["profundidade"], sim["node2vec"]
                ])

    print(f"✅ Arquivo '{caminho_saida}' gerado com sucesso.")

    # AVISO: O ARQUIVO .TSV CONTENDO O RESULTADO DAS ANÁLISES PODE DEMORAR PARA SER GERADO DEPENDENDO DA MÁQUINA UTILIZADA PARA A EXECUÇÃO!!

#### Exemplos de output do .tsv gerado após todas as  medidas aplicadas anteriormente

disciplina_a | disciplina_b | score_combinado | jaccard_pred | jaccard_succ	| profundidade | node2vec

ESZR007-21	ESZP041-14	0.24569707012536035	0.0	0.0	1.0	-0.017211719498558556

ESZR007-21	ESTS002-17	0.22492464949039911	0.0	0.0	1.0	-0.10030140203840358

ESZR007-21	ESZS001-17	0.25426921939383706	0.0	0.0	1.0	0.017076877575348332

ESZR007-21	ESZS002-17	0.2112034532523452	0.0	0.0	0.9166666666666666	-0.0718528536572858


NHH2035-13	ESTO013-17	0.26971673307139743	0.0	0.0	0.8333333333333334	0.24553359895225635

NHH2035-13	ESZG031-17	0.1975990042430288	0.0	0.0	1.0	-0.20960398302788485

NHH2035-13	MCLM002-23	0.23214671732760617	0.0	0.0	1.0	-0.07141313068957529

NHH2035-13	LHZ0040-22	0.21642313884266529	0.0	0.0	1.0	-0.13430744462933886


MCZB022-17	ESTB031-18	0.5188420281610343	0.25	0.0	0.9166666666666666	0.9087014459774706

MCZB022-17	MCBM024-23	0.47235821221947305	0.0	0.0	0.9166666666666666	0.9727661822112255

MCZB022-17	ESHP016-22	0.4825723861113853	0.0	0.0	1.0	0.9302895444455412

MCZB022-17	MCZB024-13	0.5492527035163012	0.3333333333333333	0.0	0.9166666666666666	0.9470108140652047

### Interpretação dos resultados de similaridade


##### O arquivo similaridades_disciplinas_filtrado.tsv contém pares de disciplinas que passaram por filtros estruturais (como profundidade e TPEI) e foram avaliadas segundo quatro métricas de similaridade:

jaccard_pred: Similaridade de predecessores no grafo de pré-requisitos.

jaccard_succ: Similaridade de sucessores.

profundidade: Similaridade baseada na diferença de nível curricular (quanto menor a diferença de profundidade, maior a similaridade).

node2vec: Similaridade estrutural de contexto no grafo, calculada com embeddings Node2Vec.

score_combinado: Média ponderada das quatro métricas anteriores, representando uma medida geral de similaridade entre as disciplinas.





### Interpretação dos dos números e exemplo de análise


score_combinado alto (próximo de 1) indica maior similaridade geral entre disciplinas.

node2vec acima de 0.8 sugere funções parecidas na estrutura curricular.

jaccard_pred e jaccard_succ iguais a 0 indicam ausência de pré-requisitos ou sucessores em comum.

profundidade próxima de 1 significa que estão no mesmo nível curricular.


### Exemplo

O par MCZB022-17 e MCZB024-13 teve a maior similaridade (score ≈ 0.55), com:

Jaccard de predecessores = 0.33

Profundidade ≈ 0.92

Node2Vec ≈ 0.94

Já ESZR007-21 e ESTS002-17 tiveram score baixo (≈ 0.22), pois não compartilham vizinhos e têm embeddings distantes, mesmo estando no mesmo nível.

### Visualização por meio de um mapa de calor


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Carrega o TSV
df = pd.read_csv("similaridades_disciplinas_filtrado.tsv", sep="\t")

# Seleciona 50 pares representativos
amostras = pd.concat([
    df.nlargest(10, 'score_combinado'),
    df.nsmallest(10, 'score_combinado'),
    df.nlargest(10, 'node2vec'),
    df[df['jaccard_pred'] > 0].sample(10, random_state=42),
    df.sample(10, random_state=7)
]).drop_duplicates().reset_index(drop=True)

# Indexação legível
amostras.index = amostras["disciplina_a"] + " vs " + amostras["disciplina_b"]

# Métricas para o heatmap
metricas = ["score_combinado", "jaccard_pred", "jaccard_succ", "profundidade", "node2vec"]

# Cria o heatmap
plt.figure(figsize=(14, 14))
sns.heatmap(amostras[metricas], annot=True, fmt=".2f", cmap="coolwarm", cbar=True)

plt.title("Mapa de Calor: Similaridade Entre Disciplinas (50 pares)")
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


### Output do Mapa


O gráfico abaixo apresenta um heatmap com 50 pares de disciplinas selecionados de forma representativa para demonstrar padrões nas métricas de similaridade utilizadas:

score_combinado: medida geral de similaridade calculada com base em múltiplos critérios.

jaccard_pred e jaccard_succ: medem sobreposição de pré-requisitos e sucessores, respectivamente.

profundidade: indica o quão próximas as disciplinas estão em termos de nível curricular.

node2vec: representa a semelhança estrutural entre disciplinas no grafo de currículo.

Os pares foram escolhidos para cobrir casos com alta e baixa similaridade, fortes conexões por embeddings, e exemplos aleatórios, oferecendo interpretável das relações entre disciplinas.

![alt text](mapac-1.png)